In [11]:
import pandas as pd
from IPython.display import IFrame
from bs4 import BeautifulSoup
import requests

In [12]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
IFrame(url, width=800, height=350)

In [13]:
# get data and flatten it
data = pd.read_html(url)
table = data[0]
table = table.values.flatten()

In [14]:
# remove not assigned items
filtered_data = []
for item in table:
    if "Not assigned" not in item:
        filtered_data.append(item)

In [15]:
# split and shape the data
rows = []
for item in filtered_data:
    rows.append([item[:3], item[3:item.index("(")], item[item.index("(")+1:]])

In [16]:
# convert data to pandas dataframe
df = pd.DataFrame(rows, columns=["PostalCode", "Borough", "Neighborhood"])
df["Neighborhood"] = df["Neighborhood"].str.replace(" /", ",")
df["Neighborhood"] = df["Neighborhood"].str.replace(")", "")
df["Neighborhood"] = df["Neighborhood"].str.replace("(", ",")
df["Neighborhood"] = df["Neighborhood"].str.replace(",F", ", F")
df["Neighborhood"] = df["Neighborhood"].str.replace(" ,", ", ")

df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [17]:
# load location data 
locations = pd.read_csv("https://cocl.us/Geospatial_data")
locations.head(15)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [18]:
# merge location data with dataframe
locations.columns = ["PostalCode", "Latitude", "Longitude"]
data = pd.merge(df, locations, on='PostalCode')

In [19]:
data.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don MillsNorth,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [20]:
import folium

In [21]:
latitude_toronto, longitude_toronto = 43.6534817, -79.3839347

In [22]:
map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=10)

# add markers to map
for lat, lng, borough, Neighborhood in zip(data['Latitude'], data['Longitude'], data['Borough'], data['Neighborhood']):
    label = '{}, {}'.format(Neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto